# Memory

- LLM과 주고 받은 대화를 저장하는 기능을 말한다. 
  - LLM 모델은 대화의 상태를 저장하지 않는다. 그래서 질문을 하면 그것에 대한 답변을 하고 끝이다. 대화 내용에 따라 이전 대화 내용을 바탕으로 연결되는 질문을 하고 그것에 대한 답변을 받아야 할 때가 있다. 이런 경우 지금 까지의 대화 내용을 저장하는 것을 메모리(memory)라고 한다.
- **방식**
  - 대화 내용을 저장한 뒤 다음 질문을 할 때 저장된 이전 질문들을 합쳐서 전송한다.
  - 이전 대화내용을 어떻게 저장하는지에 따라 다양한 방식의 memory 기능이 있다.
    - LLM은 [입력 토큰수의 제한이](https://platform.openai.com/docs/models) 있기 때문에 대화를 무한정 저장할 수없다.
    - Langchain은 이전 대화 내용들을 요약하거나 최신 몇개만 저장하는 방식의 다양한 memory 방식을 제공한다.

![memory.png](figures/memory.png)

## 주요 Memory Class
### ConversationBufferMemory
- https://python.langchain.com/v0.1/docs/modules/memory/types/buffer/
- **대화를 모두 그대로 저장**한다.
- 대화가 길어질 경우 저장 양이 많아지는 문제가 있다.

### ConversationBufferWindowMemory
- **최신 대화 K개만 저장** 하고 그 이전 대화는 삭제한다. (K를 window size라고 하고 객체 생성시 설정한다.)
  - 한개의 대화는 input(질문)-output(답변) 한 쌍을 말한다.
    
### ConversationTokenBufferMemory
- https://python.langchain.com/v0.1/docs/modules/memory/types/token_buffer/
- 지정한 **token 수를 넘지 않는 범위**에서 최신 대화들을 저장한다.
  

### ConversationSummaryMemory
- https://python.langchain.com/v0.1/docs/modules/memory/types/summary/
- **기존 대화들을 요약**해서 저장한다. 다만 요약은 llm이 하기 때문에 객체 생성시 llm 모델을 지정해야 한다.
- 대화 내용이 요약되어 전송되므로 토큰 수를 줄여 요금을 절약할 수있다.
  
### ConversationSummaryBufferMemory
- https://python.langchain.com/v0.1/docs/modules/memory/types/summary_buffer/
- 대화들 자체를 메모리에 저장하다가 지정한 token을 넘어가면 **오래된 대화들 순서대로 요약**한다. 
  - 최신대화는 그대로 저장하고 오래된 대화는 요약해서 저장하는 방식.

> #### ConversationSummaryMemory 토큰 비용
> - 초기 비용은 ConversationSummaryMemory가 기존 대화를 요약하기 위해 LLM을 사용하므로 ConversationBufferMemory보다 더 많은 토큰을 사용한다. 그래서 비용이 더 많이 든다.
> - 대화가 길어질수록 ConversationBufferMemory모든 대화 내용을 그대로 저장하므로 토큰 수가 선형적으로 증가한다. 
> - 반면 ConversationSummaryMemory는 대화가 길어질수록 요약된 형태로 기존 대화들이 저장되어 토큰 증가율이 더 낮아지게 되서 비용을 절감할 수있다.
> - 비용과 관련해 **짧은 대화**일 경우 ConversationBufferMemory 가 효율적이고 **긴 대화일 경우** ConversationSummaryMemory가 효율적이다.
> - **요약 비용을 절감** 하기 위해 **요약에는 저렴한 모델**을 사용할 수 있다.

> #### Deprecated
> - 위 메모리 저장 방식은 0.3.1 부터 deprecated 되었다. (1.0 버전에서 제거될 예정) 
> - 대신 RunnableWithMessageHistory 사용이 권장 된다.

### 공통 메소드
- **initializer**
  - memory_key: str = "history"
    - 대화내역을 dict로 저장하는데 그때 사용되는 key.
    - default는 **"history"**
    - Prompt template에서 memory의 대화내용을 저장하는 placeholder(template variable)의 이름을 이 memory_key 로 지정한다.
  - return_message: bool
    - True: 각 대화내용을 Message(HumanMessage, AIMessage) 객체에 저장하고 그것을 List로 묶어서 반환
    - False: 대화내용을 문자열(str)로 반환한다.
  - chat_memory: BaseChatMessageHistory
    - 대화 history를 어디에 저장할 지 설정. (메모리, sql등)
- **save_context(inputs: dict, outputs: dict)**
  - Memory에 대화내용(Context)을 저장.
  - 파라미터
    - inputs: Human message
    - outputs: AI message
- **load_memory_variables(dict)**
  - 저장된 대화내용(context)를 반환.
  - argument로 빈 dict를 넣어준다.
- **clear()**
  - 저장된 모든 대화 비우기


## 대화 저장

In [4]:
from langchain.memory import ConversationBufferMemory # 모든 대화를 다 저장.
from pprint import pprint
# 메모리 객체 생성
memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True, # 대화내용을 Message객체로 반환. False: 문자열
)

# 메모리에 대화 저장.
memory.save_context(
    inputs={"human":"안녕하세요. 은행 계좌를 개설 하고 싶습니다."}, #{role:대화내용}
    outputs={"ai":"계좌 개설을 원하시는 군요. 신분증을 먼저 준비하세요."}
)
# 저장된 대화들을 조회
save_conv = memory.load_memory_variables({})
pprint(save_conv)
# 저장형식: dictionary
### {memory_key: "대화들"}

{'chat_history': [HumanMessage(content='안녕하세요. 은행 계좌를 개설 하고 싶습니다.', additional_kwargs={}, response_metadata={}),
                  AIMessage(content='계좌 개설을 원하시는 군요. 신분증을 먼저 준비하세요.', additional_kwargs={}, response_metadata={})]}


In [5]:
# save_conv['history']
save_conv['chat_history']

[HumanMessage(content='안녕하세요. 은행 계좌를 개설 하고 싶습니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='계좌 개설을 원하시는 군요. 신분증을 먼저 준비하세요.', additional_kwargs={}, response_metadata={})]

In [6]:
memory.save_context(
    inputs={"human":"신분증을 준비했습니다. 다음에는 무엇을 하면 되나요?"},
    outputs={"ai":"신분증 앞면을 촬영해서 업로드 해주세요."}
)
pprint(memory.load_memory_variables({}))

{'chat_history': [HumanMessage(content='안녕하세요. 은행 계좌를 개설 하고 싶습니다.', additional_kwargs={}, response_metadata={}),
                  AIMessage(content='계좌 개설을 원하시는 군요. 신분증을 먼저 준비하세요.', additional_kwargs={}, response_metadata={}),
                  HumanMessage(content='신분증을 준비했습니다. 다음에는 무엇을 하면 되나요?', additional_kwargs={}, response_metadata={}),
                  AIMessage(content='신분증 앞면을 촬영해서 업로드 해주세요.', additional_kwargs={}, response_metadata={})]}


In [7]:
memory.clear() # 저장내용 모두 삭제
pprint(memory.load_memory_variables({}))

{'chat_history': []}


# 주요 메모리 클래스 예제

## ConversationBufferMemory
메모리에 대화를 모두 저장한다.

In [11]:
from langchain.memory import ConversationBufferMemory
from pprint import pprint

memory = ConversationBufferMemory(
    return_messages=True
)

memory.save_context(
    inputs={
        "human":"안녕하세요"
    },
    outputs={
        "ai": "반갑습니다."
    }
)

memory.save_context(
    inputs={
        "human":"한국 여행에 대해 물어보려고 합니다."
    },
    outputs={
        "ai":"무엇이든 물어보세요."
    }
)

memory.save_context(
    inputs={
        "human":"여행 할 때 꼭 먹어봐야할 음식 3개만 추천해줘."
    },
    outputs={
        "ai":"불고기, 비빔밥, 삼겹살을 추천합니다."
    }
)

memory.save_context(
    inputs={
        "human":"여행지 두 곳을 추천해줘."
    },
    outputs={
        "ai":"경복궁과 국립중앙박물관입니다."
    }
)
memory.save_context(
    inputs={
        "human":"여행지 한 곳 더 추천해줘."
    },
    outputs={
        "ai":"민속촌을 추천합니다."
    }
)

memory.save_context(
    inputs={
        "human":"한국의 자연을 느낄 수있는 여행지를 알려줘."
    },
    outputs={
        "ai":"북한산을 추천합니다. 서울 도심에서 자연을 느낄 수있습니다."
    }
)

pprint(memory.load_memory_variables({}))

{'history': [HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}),
             AIMessage(content='반갑습니다.', additional_kwargs={}, response_metadata={}),
             HumanMessage(content='한국 여행에 대해 물어보려고 합니다.', additional_kwargs={}, response_metadata={}),
             AIMessage(content='무엇이든 물어보세요.', additional_kwargs={}, response_metadata={}),
             HumanMessage(content='여행 할 때 꼭 먹어봐야할 음식 3개만 추천해줘.', additional_kwargs={}, response_metadata={}),
             AIMessage(content='불고기, 비빔밥, 삼겹살을 추천합니다.', additional_kwargs={}, response_metadata={}),
             HumanMessage(content='여행지 두 곳을 추천해줘.', additional_kwargs={}, response_metadata={}),
             AIMessage(content='경복궁과 국립중앙박물관입니다.', additional_kwargs={}, response_metadata={}),
             HumanMessage(content='여행지 한 곳 더 추천해줘.', additional_kwargs={}, response_metadata={}),
             AIMessage(content='민속촌을 추천합니다.', additional_kwargs={}, response_metadata={}),
             HumanMessage(content='한국의 

In [ ]:
# LLM 모델에 요청
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4o-mini")
prompt_template = ChatPromptTemplate(
    [
        ("system", "당신은 한국 여행 전문가 입니다. 한국 여행과 관련된 다양한 정보를 알려주세요."),
        MessagesPlaceholder("history", optional=True),
        ("human", "{query}")
    ]
)
chain = prompt_template | model | StrOutputParser()

In [ ]:
query = "마지막 소개한 여행지를 가는 방법에 대해 알려주세요."
res = chain.invoke(
    {"query":query, "history":memory.load_memory_variables({})["history"]}
)

In [17]:
print(res)

어떤 여행지를 말씀하시는지 구체적으로 알려주시면, 그 여행지를 가는 방법에 대해 자세히 안내해드리겠습니다. 한국에는 다양한 여행지가 있으니, 특정한 장소를 지정해 주시면 더욱 정확한 정보를 제공할 수 있습니다. 예를 들어, 서울, 부산, 제주도 등 특정 지역을 말씀해 주시면 좋습니다!


In [ ]:
## 최신 대화를 memery에 저장
memory.save_context(
    inputs={"human":query},
    outputs={"ai":res}
)

In [19]:
pprint(memory.load_memory_variables({})["history"])

[HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='반갑습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='한국 여행에 대해 물어보려고 합니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='무엇이든 물어보세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='여행 할 때 꼭 먹어봐야할 음식 3개만 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='불고기, 비빔밥, 삼겹살을 추천합니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='여행지 두 곳을 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='경복궁과 국립중앙박물관입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='여행지 한 곳 더 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='민속촌을 추천합니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='한국의 자연을 느낄 수있는 여행지를 알려줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='북한산을 추천합니다. 서울 도심에서 자연을 느낄 수있습니다.', addition

In [ ]:
def save_memory(memory):
    memory.save_context(
        inputs={
            "human":"안녕하세요"
        },
        outputs={
            "ai": "반갑습니다."
        }
    )

    memory.save_context(
        inputs={
            "human":"한국 여행에 대해 물어보려고 합니다."
        },
        outputs={
            "ai":"무엇이든 물어보세요."
        }
    )

    memory.save_context(
        inputs={
            "human":"여행 할 때 꼭 먹어봐야할 음식 3개만 추천해줘."
        },
        outputs={
            "ai":"불고기, 비빔밥, 삼겹살을 추천합니다."
        }
    )

    memory.save_context(
        inputs={
            "human":"여행지 두 곳을 추천해줘."
        },
        outputs={
            "ai":"경복궁과 국립중앙박물관입니다."
        }
    )
    memory.save_context(
        inputs={
            "human":"여행지 한 곳 더 추천해줘."
        },
        outputs={
            "ai":"민속촌을 추천합니다."
        }
    )

    memory.save_context(
        inputs={
            "human":"한국의 자연을 느낄 수있는 여행지를 알려줘."
        },
        outputs={
            "ai":"북한산을 추천합니다. 서울 도심에서 자연을 느낄 수있습니다."
        }
    )

## ConversationBufferWindowMemory
최신 대화 K개만 저장 한다.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from warnings import filterwarnings
filterwarnings('ignore')  # 경고메세지는 출력안되게 처리.

memory = ConversationBufferWindowMemory(
    k=5, # 최근 대화(input+output: 1개) k(3) 개까지만 저장. 오래된 것은 삭제.
    return_messages=True
)
# memory.save_context(inputs=...., outputs=...)
save_memory(memory)

In [ ]:
from pprint import pprint
pprint(memory.load_memory_variables({})['history'])

[HumanMessage(content='한국 여행에 대해 물어보려고 합니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='무엇이든 물어보세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='여행 할 때 꼭 먹어봐야할 음식 3개만 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='불고기, 비빔밥, 삼겹살을 추천합니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='여행지 두 곳을 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='경복궁과 국립중앙박물관입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='여행지 한 곳 더 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='민속촌을 추천합니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='한국의 자연을 느낄 수있는 여행지를 알려줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='북한산을 추천합니다. 서울 도심에서 자연을 느낄 수있습니다.', additional_kwargs={}, response_metadata={})]


## ConversationTokenBufferMemory
대화의 토큰 길이를 기준으로 최신 대화만 저장한다.

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o-mini')
memory = ConversationTokenBufferMemory(
    llm=model, # llm 모델이 필요. (토큰 체크용.-모델마다 토큰 계산이 다르기 때문에 사용할 모델을 넣어준다.)
    max_token_limit=100, 
    return_messages=True
)
save_memory(memory)

In [41]:
pprint(memory.load_memory_variables({})["history"])

[AIMessage(content='경복궁과 국립중앙박물관입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='여행지 한 곳 더 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='민속촌을 추천합니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='한국의 자연을 느낄 수있는 여행지를 알려줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='북한산을 추천합니다. 서울 도심에서 자연을 느낄 수있습니다.', additional_kwargs={}, response_metadata={})]


## ConversationSummaryMemory
대화를 요약해서 저장한다.

In [ ]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(
    llm=model, # 대화내용을 요약할 모델을 설정. 
)
save_memory(memory)

In [43]:
pprint(memory.load_memory_variables({})['history'])

('The human greets the AI in Korean, and the AI responds with a greeting. The '
 'human expresses that they want to ask about traveling to Korea, and the AI '
 'invites them to ask anything. The human requests the AI to recommend three '
 'must-try foods while traveling, and the AI suggests bulgogi, bibimbap, and '
 'samgyeopsal. The human then asks the AI to recommend two travel '
 'destinations, and the AI suggests Gyeongbokgung Palace and the National '
 'Museum of Korea. Finally, the human asks for one more travel destination, '
 'and the AI recommends the Korean Folk Village. The human then asks for a '
 "travel destination to experience Korea's nature, and the AI recommends "
 'Bukhansan, which allows visitors to enjoy nature within the city of Seoul.')


In [ ]:
print(memory.prompt) # LLM에게 대화를 요청하기 위한 prompt template 조회.
print("--------------------------")
print(memory.prompt.template)

input_variables=['new_lines', 'summary'] input_types={} partial_variables={} template='Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\nEND OF EXAMPLE\n\nCurrent summary:\n{summary}\n\nNew lines of conversation:\n{new_lines}\n\nNew summary:'
--------------------------
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current 

## ConversationSummaryBufferMemory
오래된 대화는 요약하고 최신대화는 그대로 저장한다.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=model, #요약에 사용할 모델.
    max_token_limit=100, 
    return_messages=True
)
save_memory(memory)

In [48]:
pprint(memory.load_memory_variables({})['history'])

[SystemMessage(content='The human greets the AI in Korean, and the AI responds warmly. The human then states that they would like to ask about traveling in Korea, and the AI encourages the human to ask anything. The human requests three must-try foods when traveling, and the AI recommends bulgogi, bibimbap, and samgyeopsal. The human then asks for two travel destination recommendations.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='경복궁과 국립중앙박물관입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='여행지 한 곳 더 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='민속촌을 추천합니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='한국의 자연을 느낄 수있는 여행지를 알려줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='북한산을 추천합니다. 서울 도심에서 자연을 느낄 수있습니다.', additional_kwargs={}, response_metadata={})]


# Chain
## Off-the-shelf chains

In [83]:
from langchain.chains import LLMChain, ConversationChain # 미리 정의된 Chain클래스들(off the shelf chain)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint
from warnings import filterwarnings
from dotenv import load_dotenv

filterwarnings('ignore')
load_dotenv()

True

In [84]:
prompt_template = ChatPromptTemplate(
    [
        ("system", "답변은 세 문장 이하로 간결하게 대답해줘."),
        MessagesPlaceholder("history"), # varable_name -> memory_key와 동일한 값으로 지정.
        ("human", "{query}")
    ]
)
# 모델: 대화기록을 summary하는 모델과 실제 요청에 응답하는 모델을 따로 생성할 수있다.
s_model = ChatOpenAI(
    model="gpt-4o-mini"
)
# 메모리 객체
memory = ConversationSummaryBufferMemory(
    llm=s_model,
    max_token_limit=200,
    # memory_key="history", # default: history
    return_messages=True
)
output_parser = StrOutputParser()
model = ChatOpenAI(
    model="gpt-4o-mini"
)
chain = LLMChain(
    llm=model, # LLM Model
    prompt=prompt_template,
    output_parser=output_parser,
    memory=memory  # 메모리
)

In [85]:
result = chain.invoke({"query":"안녕하세요"})
# result - template 변수들에 넣은 값들 key=valaue, history에 저장된 값들, 응답값들을
#          딕셔너리에 묶어서 반환.
print(result)

{'query': '안녕하세요', 'history': [HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={})], 'text': '안녕하세요! 어떻게 도와드릴까요?'}


In [78]:
print(result["text"])

안녕하세요! 어떻게 도와드릴까요?


In [79]:
# 메모리에 저장된 내역 확인.
memory.load_memory_variables({})["history"]

[HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={})]

In [86]:
query = input("질문:")
while True:
    if query == "!q":
        break
    # 모델에 질문
    result = chain.invoke({"query":query})
    print("Human:", query)
    print("AI:", result['text'])
    print("=========================================================")
    # 다음 질문 입력
    query = input("질문:")

Human: 내 이름은 김성환입니다.
AI: 반갑습니다, 김성환님! 무엇을 도와드릴까요?
Human: 내 이름을 불러줘.
AI: 김성환님!
Human: 한국음식 3개를 목록형태로 알려줘.
AI: 1. 김치찌개  
2. 비빔밥  
3. 불고기
Human: 두번째 음식의 요리법을 알려줘.
AI: 비빔밥 요리법:

1. 밥을 준비하고, 시금치, 당근, 버섯, 콩나물 등 각종 나물을 볶아 준비합니다.
2. 그릇에 밥을 담고, 볶은 나물과 고기를 고명으로 올립니다.
3. 고추장과 참기름을 곁들여 잘 비비고, 계란 후라이를 올려서 완성합니다.
Human: .
AI: 무엇을 도와드릴까요?


In [58]:
pprint(memory.load_memory_variables({})['history'])

[SystemMessage(content='The human greets the AI, which responds warmly and asks how it can assist. The human inquires about three memory classes in LangChain that remember conversation content. The AI explains there are three types: \n\n1. **ConversationBufferMemory**, which sequentially stores all inputs and outputs to maintain conversation context.\n2. **ConversationSummaryMemory**, which summarizes and stores key information from the conversation for efficient memory management, especially in longer dialogues.\n3. **CombinedMemory**, which integrates multiple memory classes to retain both detailed and summarized information for a richer conversation experience.\n\nThese memory classes help LangChain\'s conversational AI effectively remember and generate responses during interactions. The human then asks the AI to explain the second class, and the AI responds by clarifying that "second class" can vary in meaning based on context. Upon the human\'s further request in Korean for an exp

## LCEL

In [93]:
from langchain_core.runnables import RunnablePassthrough, chain

prompt_template = ChatPromptTemplate(
    [
        ("system", "답변은 세 문장 이하로 간결하게 대답해줘."),
        MessagesPlaceholder("history"), # varable_name -> memory_key와 동일한 값으로 지정.
        ("human", "{query}")
    ]
)

model = ChatOpenAI(model='gpt-4o-mini')
output_parser = StrOutputParser()
memory = ConversationSummaryBufferMemory(
    llm=model,
    max_token_limit=200,
    return_messages=True,
    memory_key="history"
)

In [ ]:
def load_memory(input):
    # RunnablePassThrough.assign(key=함수) 에 넣어줄 함수.
    # input: RunnablePassThrough.invoke(입력값) 의 입력값을 받는 파라미터.
    ##### memory에서 저장된 대화 내역을 반환하는 함수.
    return memory.load_memory_variables({})["history"]

@chain
def question_chain(query):
    # chain.invoke({"query":query})
    # 입력: {"query":query} -> {"query":query, "history":load_memory()}
    ### 메모리의 history를 추가해서 프롬프트에 전달 (RunnablePassThrough.assign())
    chain = (RunnablePassthrough.assign(history=load_memory) 
            | prompt_template
            | model
            | output_parser)
    # chain을 이용해 질문 요청 -> 응답
    result = chain.invoke({"query":query})
    # 질문과 응답을 메모리에 저장. - 문자열로 저장.
    memory.save_context(inputs={"human":query}, outputs={"ai":result})
    return result

In [ ]:
result = question_chain.invoke("한국의 유명한 위인 세명을 알려주세요.")

In [100]:
print(result)

세종대왕, 이순신, 김구가 한국의 유명한 위인입니다.


In [101]:
result = question_chain.invoke("두번째 사람은 언제 태어났나?")
print(result)

이순신은 1545년 4월 28일에 태어났습니다.
